## Environment Setup

In [ ]:
! pip install langchain_community langchain_mistralai langchainhub langchain tiktoken chromadb

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<your LangChain API KEY goes here>'
os.environ['MISTRAL_API_KEY'] = '<your Mistral API KEY goes here>'
os.environ['HF_TOKEN'] = '<your Hugging Face TOKEN goes here>'


In [ ]:
questions = [ "What does Moby Dick say about humanity's struggle against nature?",
              "How does Moby Dick explore the theme of obsession through Ahab's quest?",
              "How is Captain Ahab portrayed as both a hero and a villain in Moby Dick?",
              # "What motivates Ishmael to join the Pequod, and how does he change throughout the novel?",
              # "What does the white whale symbolize in Moby Dick, and how does it relate to Ahab's obsession?",
              # "How does the novel Moby Dick use the ocean as a symbol of the unknown?",
              # "How does Melville’s narrative style in Moby Dick contribute to the sense of adventure and mystery?",
              # "How does Ishmael’s perspective shape the reader’s understanding of the story in Moby Dick?",
              # "How does Moby Dick reflect 19th-century views on fate and destiny?",
              # "What philosophical questions does Melville raise about human existence and purpose in Moby Dick?",
              # "How does Melville use imagery to depict the sea as both beautiful and terrifying in Moby Dick?",
              # "How does Moby Dick describe the vastness and danger of the open sea?",
              # "What moral dilemmas do the crew members face in Moby Dick?",
              # "How does Moby Dick present Ahab's pursuit of revenge as both justified and self-destructive?"
              ]

question = questions[0]

## Indexing

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
# there can be mulitple urls
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/2701/pg2701.txt")
books = loader.load()

### Splitting the text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(books)
len(splits)


### vectorize chunks

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=MistralAIEmbeddings())

## Retrieval

### Init Retriever

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

### Sub-questions Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are a helpful assistant designed to generate multiple sub-questions related to an input question.
Your goal is to break down the main question into distinct sub-problems that can be answered individually.
Generate three related search queries based on: {question} and output each query on a new line."""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

generate_queries = (
    prompt_perspectives
    | ChatMistralAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

### Output original questions and their detailized sub-questions

In [ ]:
for question in questions:
    print("-------------")
    print(question)
    sub_problems = generate_queries.invoke({"question":question})
    for sub_problem in sub_problems:
        print("\t"+sub_problem)

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

# Generation

In [ ]:
from operator import itemgetter
from langchain_mistralai import ChatMistralAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatMistralAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

for question in questions:
    print("-------------")
    print(question)
    rephrased_questions = generate_queries.invoke({"question":question})
    for rephrased_question in rephrased_questions:
        print("\t"+rephrased_question)
    print("\nAnswer: ")
    print(final_rag_chain.invoke({"question":question}))